In [61]:
import requests
from bs4 import BeautifulSoup
import typing as tp
from tqdm.notebook import tqdm 
import time

def get_soup(url: str) -> BeautifulSoup:
    """Get page content and create html tree using bs4"""
    res = requests.get(url)
    return BeautifulSoup(res.content, 'html.parser')


def get_main_hrefs(tree: BeautifulSoup) -> tp.List[tp.Dict]:
    books = tree.find_all('li', {'class': 'col-xs-6 col-sm-4 col-md-3 col-lg-3'})
    return [{
        'href': item.h3.a.get('href'), 
        'title': item.h3.a.get('title')
    } for item in books]


MAIN_URL = "https://books.toscrape.com/catalogue/"

data = [ ]
for p in tqdm(range(1, 51)):
    url = MAIN_URL + f'page-{p}.html'
    tree = get_soup(url)
    books = get_main_hrefs(tree)
    data.extend(books)
    time.sleep(0.002)   # код ничего не делает 0.2 секунды

  0%|          | 0/50 [00:00<?, ?it/s]

In [58]:
import pandas as pd
df = pd.DataFrame(data)
df.head()

,href,title
0,a-light-in-the-attic_1000/index.html,A Light in the Attic
1,tipping-the-velvet_999/index.html,Tipping the Velvet
2,soumission_998/index.html,Soumission
3,sharp-objects_997/index.html,Sharp Objects
4,sapiens-a-brief-history-of-humankind_996/index...,Sapiens: A Brief History of Humankind


In [82]:
def get_book_info(infa):
    href = MAIN_URL + infa['href']
    tree = get_soup(href)
    table = tree.find('table', {'class': "table table-striped"})
    table = pd.read_html(str(table))[0].set_index(0).to_dict()[1]
    price = tree.find('p', {'class': 'price_color'}).text

    infa.update(table)
    infa.update({'price': price})
    return infa

In [84]:
updated_data = [ ]
for item in data[:10]:
    try:
        updated_data.append(get_book_info(data[1]))
    except:
        print('Не получилось', item['href'])

In [85]:
dff = pd.DataFrame(updated_data)
dff.head()

,href,title,UPC,Product Type,Price (excl. tax),Price (incl. tax),Tax,Availability,Number of reviews,price
0,tipping-the-velvet_999/index.html,Tipping the Velvet,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,£53.74
1,tipping-the-velvet_999/index.html,Tipping the Velvet,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,£53.74
2,tipping-the-velvet_999/index.html,Tipping the Velvet,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,£53.74
3,tipping-the-velvet_999/index.html,Tipping the Velvet,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,£53.74
4,tipping-the-velvet_999/index.html,Tipping the Velvet,90fa61229261140a,Books,£53.74,£53.74,£0.00,In stock (20 available),0,£53.74
